In [1]:
import pandas as pd 
import numpy as np

In [2]:
weather_data = pd.read_csv('./data/weather_data4.csv')
bike_data = pd.read_csv('./data/combined_csv.csv')

In [3]:
weather_data.head()

,date,time,weather_description,temp_in_f,humidity_in_%,pressure,precipitation_in_inches,visibility_in_miles,wind_direction,wind_speed_in_mph,temp_in_f_delta,pressure_delta,humidity_delta,visibility_delta,precipitation_delta,wind_speed_delta
0,2017-12-31,00:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.0,N,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-12-31,01:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.0,N,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2017-12-31,02:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.3,N,0.0,0.0,0.0,0.0,0.3,0.0,0.0
3,2017-12-31,03:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.3,N,0.0,0.0,0.0,0.0,0.3,0.0,0.0
4,2017-12-31,04:00:00,Clear,33.0,100.0,30.0,0.0,9.0,N,0.0,1.0,0.0,0.0,5.0,0.0,0.0


In [4]:
bike_data.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,552,2018-01-01 00:05:06,2018-01-01 00:14:18,31104,Adams Mill & Columbia Rd NW,31400,Georgia & New Hampshire Ave NW,W00886,Member
1,1282,2018-01-01 00:14:30,2018-01-01 00:35:53,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W01435,Casual
2,1265,2018-01-01 00:14:53,2018-01-01 00:35:58,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W21242,Casual
3,578,2018-01-01 00:15:31,2018-01-01 00:25:09,31406,14th & Upshur St NW,31103,16th & Harvard St NW,W21322,Casual
4,372,2018-01-01 00:18:02,2018-01-01 00:24:15,31618,4th & East Capitol St NE,31619,Lincoln Park / 13th & East Capitol St NE,W00119,Member


In [8]:
weather_data['time'] = weather_data['time'].astype('str') 
weather_data['date'] = weather_data['date'].astype('str') 
weather_data.dtypes

date                        object
time                        object
weather_description         object
temp_in_f                  float64
humidity_in_%              float64
pressure                   float64
precipitation_in_inches    float64
visibility_in_miles        float64
wind_direction              object
wind_speed_in_mph          float64
temp_in_f_delta            float64
pressure_delta             float64
humidity_delta             float64
visibility_delta           float64
precipitation_delta        float64
wind_speed_delta           float64
dtype: object

## Cleaning the data

In [5]:
## Removing unwanted characters such as %, F etc (i.e units)
weather_data['temp_in_f'] = weather_data['temp_in_f'].str.extract('(\d+)', expand=False)
weather_data['dewpt_in_f'] = weather_data['dewpt_in_f'].str.extract('(\d+)', expand=False)
weather_data['humidity_in_%'] = weather_data['humidity_in_%'].str.extract('(\d+)', expand=False)
weather_data['visibility_in_miles'] = weather_data['visibility_in_miles'].str.extract('(\d+)', expand=False)
weather_data['precipitation_in_inches'] = weather_data['precipitation_in_inches'].str.extract('(\d+)', expand=False)

In [9]:
## convert to datetime 
bike_data['Start date'] = pd.to_datetime(bike_data['Start date'])
bike_data['End date'] = pd.to_datetime(bike_data['End date'])

In [10]:
## Separating date and time
bike_data['start_date'] = bike_data['Start date'].dt.date
bike_data['end_date'] = bike_data['End date'].dt.date
bike_data['start_time'] = bike_data['Start date'].dt.time
bike_data['end_time'] = bike_data['End date'].dt.time

In [11]:
# get duration in minutes 
bike_data['duration_in_mins'] = (bike_data['End date'] - bike_data['Start date']) / np.timedelta64(1,'m') 

In [12]:
bike_data = bike_data.rename(columns={'Start station number': 'start_station_id', 'Start station': 'start_station', 
                         'End station number': 'end_station_id', 'End station':'end_station', 
                         'Bike number': 'bike_id', 'Member type': 'member_type'})

## Sanity checks 

In [13]:
# checking number of nulls 
weather_data['time'].value_counts()

15:00:00    365
08:00:00    365
06:00:00    365
19:00:00    365
14:00:00    365
05:00:00    365
12:00:00    365
17:00:00    365
20:00:00    365
09:00:00    365
04:00:00    365
13:00:00    365
10:00:00    365
16:00:00    365
22:00:00    365
03:00:00    365
18:00:00    365
07:00:00    365
21:00:00    365
11:00:00    365
01:00:00    364
23:00:00    364
00:00:00    363
02:00:00    363
nan           2
Name: time, dtype: int64

## Merge weather data and bike data

In [14]:
# get nearest hour
bike_data['nearest_date'] = bike_data['Start date'].dt.round(freq = 'H')
bike_data['date'] = bike_data['nearest_date'].dt.date
bike_data['date'] = bike_data['date'].apply(lambda x:x.strftime('%Y-%m-%d'))
bike_data['time'] = bike_data['nearest_date'].dt.time
bike_data['time'] = bike_data['time'].apply(lambda x:x.strftime('%H:%M:%S'))

In [15]:
#weather_data['time'] = weather_data['time'].apply(lambda x: x.strip())
#weather_data['date'] = weather_data['date'].apply(lambda x: x.strip())

In [16]:
# merge
final_data = pd.merge(bike_data, weather_data, on=['date', 'time'], how='inner')

In [17]:
# drop columns 
final_data = final_data.drop(['nearest_date', 'Duration', 'Start date', 'End date', 
                           'date', 'time'], axis = 1)

In [18]:
bike_data.shape

(3542684, 17)

In [19]:
final_data.shape

(3534573, 25)

In [20]:
# checking number of nulls 
final_data.isna().sum() 

start_station_id              0
start_station                 0
end_station_id                0
end_station                   0
bike_id                       0
member_type                   0
start_date                    0
end_date                      0
start_time                    0
end_time                      0
duration_in_mins              0
weather_description        1453
temp_in_f                     0
humidity_in_%                 0
pressure                      0
precipitation_in_inches       0
visibility_in_miles        4948
wind_direction                0
wind_speed_in_mph            61
temp_in_f_delta              94
pressure_delta               94
humidity_delta               94
visibility_delta           6215
precipitation_delta          94
wind_speed_delta            155
dtype: int64

In [30]:
final_data[final_data['visibility_in_miles'].isna()]

,start_station_id,start_station,end_station_id,end_station,bike_id,member_type,start_date,end_date,start_time,end_time,...,precipitation_in_inches,visibility_in_miles,wind_direction,wind_speed_in_mph,temp_in_f_delta,pressure_delta,humidity_delta,visibility_delta,precipitation_delta,wind_speed_delta
2504734,31280,11th & S St NW,31411,Georgia & Missouri Ave NW,W21211,Casual,2018-09-02,2018-09-02,04:30:37,05:16:46,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504735,31008,12th & Army Navy Dr,31003,Crystal Dr & 15th St S,W20454,Member,2018-09-02,2018-09-02,04:32:21,04:34:52,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504736,31291,Vermont Ave & I St NW,31276,15th & L St NW,W01337,Member,2018-09-02,2018-09-02,04:33:27,04:35:38,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504737,31619,Lincoln Park / 13th & East Capitol St NE,31512,Neal St & Trinidad Ave NE,W21020,Member,2018-09-02,2018-09-02,04:34:00,04:40:38,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504738,31000,Eads St & 15th St S,31006,S Joyce & Army Navy Dr,W20316,Member,2018-09-02,2018-09-02,04:35:51,04:41:15,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504739,31518,New York Ave & Hecht Ave NE,31227,13th St & New York Ave NW,W21454,Member,2018-09-02,2018-09-02,04:37:08,05:00:43,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504740,31260,23rd & E St NW,31204,20th & E St NW,W20489,Member,2018-09-02,2018-09-02,04:38:59,04:41:24,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504741,31223,Convention Center / 7th & M St NW,31600,5th & K St NW,W20543,Member,2018-09-02,2018-09-02,04:40:36,07:49:51,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504742,31101,14th & V St NW,31296,Columbia & Ontario Rd NW,W21490,Member,2018-09-02,2018-09-02,04:43:41,04:58:09,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2504743,31204,20th & E St NW,31260,23rd & E St NW,W00978,Member,2018-09-02,2018-09-02,04:50:11,04:53:22,...,0.0,NaN,N,0.0,0.0,0.0,0.0,NaN,0.0,0.0


In [26]:
final_data.tail()

,start_station_id,start_station,end_station_id,end_station,bike_id,member_type,start_date,end_date,start_time,end_time,...,precipitation_in_inches,visibility_in_miles,wind_direction,wind_speed_in_mph,temp_in_f_delta,pressure_delta,humidity_delta,visibility_delta,precipitation_delta,wind_speed_delta
3534568,31045,Commerce St & Fayette St,31041,Prince St & Union St,W00399,Casual,2018-12-31,2018-12-31,23:28:16,23:43:22,...,0.0,10.0,SSW,11.0,6.0,0.0,-14.0,0.0,0.0,5.3
3534569,31041,Prince St & Union St,31903,Royal & Wilkes St,W00490,Member,2018-12-31,2018-12-31,23:28:16,23:32:50,...,0.0,10.0,SSW,11.0,6.0,0.0,-14.0,0.0,0.0,5.3
3534570,31045,Commerce St & Fayette St,31041,Prince St & Union St,W22125,Member,2018-12-31,2018-12-31,23:28:23,23:43:32,...,0.0,10.0,SSW,11.0,6.0,0.0,-14.0,0.0,0.0,5.3
3534571,31602,Park Rd & Holmead Pl NW,31291,Vermont Ave & I St NW,W23745,Member,2018-12-31,2018-12-31,23:28:50,23:50:09,...,0.0,10.0,SSW,11.0,6.0,0.0,-14.0,0.0,0.0,5.3
3534572,31125,15th & W St NW,31200,Massachusetts Ave & Dupont Circle NW,W00436,Member,2018-12-31,2018-12-31,23:29:55,23:36:24,...,0.0,10.0,SSW,11.0,6.0,0.0,-14.0,0.0,0.0,5.3


In [27]:
# write out data file 
final_data.to_csv('./data/data.csv', index=False)